In [ ]:
import boto3
import pandas as pd
import json
import io 
import os
import time
import redis
from apscheduler.schedulers.blocking import BlockingScheduler
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#urls end keys
s3 = boto3.client(
        "s3",
        endpoint_url=os.environ.get("S3_ENDPOINT_URL"),
        aws_access_key_id=os.environ.get("MINIO_ROOT_USER"),
        aws_secret_access_key=os.environ.get("MINIO_ROOT_PASSWORD"),
    )
    

In [ ]:
def get_data():
    current_time = datetime.now()
    path = f"year={current_time.year:04d}/month={current_time.month:02d}/day={current_time.day:02d}/hour={current_time.hour-1:02d}"
    response = s3.list_objects(Bucket="test-bucket", Prefix=path)
    dataframes = []

    for obj in response["Contents"]:
        file_key = obj["Key"]
        if file_key.endswith(".csv"):
            file_obj = s3.get_object(Bucket="test-bucket", Key=obj["Key"])
            df = pd.read_csv(file_obj["Body"])
            dataframes.append(df)
            
    # Additional processing steps
    processed_dataframes = []
    for df in dataframes:
        # Process each dataframe as needed
        processed_df = process_dataframe(df)
        processed_dataframes.append(processed_df)
    
    return processed_dataframes


def process_dataframe(df):
    # Example processing: calculate the sum of each column
    column_sums = df.sum()
    processed_df = pd.DataFrame({"Column Sums": column_sums}).transpose()
    return processed_df


In [ ]:
get_data()

In [ ]:
def get_data_from_postgress():
    bookstore_data = pd.read_csv("../data/bookstore.csv")
    final_df = pd.merge(dfs, bookstore_data, on='ISBN-10', how='inner')
    return final_df


In [ ]:
get_data_from_postgress()

In [ ]:


def load_to_redis():
    redis_client = redis.Redis(host="localhost", port=6379, db=0)
    redis_key = "minio_files"
    redis_output = get_data_from_postgress()
    
    # Convert dataframe to JSON
    redis_data = redis_output.to_json(orient="records")
    
    # Store JSON data in Redis
    redis_client.set(redis_key, redis_data)
    
    return redis_output



In [ ]:
def transformed_data():
    transformed_df = load_to_redis()
    transformed_df_1 = transformed_df.groupby(["Format", "Language"]).agg({"Price": ["count", "sum"]})
    transformed_df_1.columns = ["number_of_books_sold", "sale_values"]
    return transformed_df, transformed_df_1

result_df, result_df_1 = transformed_data()


In [ ]:
transformed_df, transformed_df_1 = transformed_data()

# Calculate percentage of users who buy books in Spanish
users = transformed_df["user_id"].nunique()
spanish_language_users = transformed_df[transformed_df.Language == "Spanish"]["user_id"].nunique()
spanish_book_percentage = (spanish_language_users / users) * 100

print(f"Spanish Book Purchased by users in Percentage: {spanish_book_percentage:.3f}%")


In [26]:
if __name__ == "_main_":
    scheduler = BlockingScheduler()
    scheduler.add_job(get_data(), 'interval', minutes=60)
    scheduler.start()